参考：

作者同一人。

1、https://blog.csdn.net/xiaosa_kun/article/details/84868403

2、https://www.jianshu.com/p/0536182dbb4d

主要工作：


1、连续变量分箱处理，然后合并到连续变量中；

2、剔除离群点。

3、使用模型：`XGBRegressor`。

In [1]:
import numpy as np
import pandas as pd

data_train = pd.read_csv('../input/train.csv')
data_train.head()

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9


In [2]:
data_train.shape

(333, 15)

In [3]:
X = data_train.drop(['ID', 'medv'], axis=1)
X.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33
4,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43


In [4]:
field_cut = {
    'crim': [0, 10, 20, 100],
    'zn': [-1, 5, 18, 20, 40, 80, 86, 100],
    'indus': [-1, 7, 15, 23, 40],
    'nox': [0, 0.51, 0.6, 0.7, 0.8, 1],
    'rm': [0, 4, 5, 6, 7, 8, 9],
    'age': [0, 60, 80, 100],
    'dis': [0, 2, 6, 14],
    'rad': [0, 5, 10, 25],
    'tax': [0, 200, 400, 500, 800],
    'ptratio': [0, 14, 20, 23],
    'black': [0, 100, 350, 450],
    'lstat': [0, 5, 10, 20, 40]
}

In [5]:
def dataProcessing(df):
    cut_df = pd.DataFrame()
    for field in field_cut.keys():
        cut_series = pd.cut(df[field], field_cut[field], right=True)
        onehot_df = pd.get_dummies(cut_series, prefix=field)
        cut_df = pd.concat([cut_df, onehot_df], axis=1)
    new_df = pd.concat([df, cut_df], axis=1)
    return new_df


X_new = dataProcessing(X)

In [6]:
pd.options.display.max_columns = 100

X_new.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,"crim_(0, 10]","crim_(10, 20]","crim_(20, 100]","zn_(-1, 5]","zn_(5, 18]","zn_(18, 20]","zn_(20, 40]","zn_(40, 80]","zn_(80, 86]","zn_(86, 100]","indus_(-1, 7]","indus_(7, 15]","indus_(15, 23]","indus_(23, 40]","nox_(0.0, 0.51]","nox_(0.51, 0.6]","nox_(0.6, 0.7]","nox_(0.7, 0.8]","nox_(0.8, 1.0]","rm_(0, 4]","rm_(4, 5]","rm_(5, 6]","rm_(6, 7]","rm_(7, 8]","rm_(8, 9]","age_(0, 60]","age_(60, 80]","age_(80, 100]","dis_(0, 2]","dis_(2, 6]","dis_(6, 14]","rad_(0, 5]","rad_(5, 10]","rad_(10, 25]","tax_(0, 200]","tax_(200, 400]","tax_(400, 500]","tax_(500, 800]","ptratio_(0, 14]","ptratio_(14, 20]","ptratio_(20, 23]","black_(0, 100]","black_(100, 350]","black_(350, 450]","lstat_(0, 5]","lstat_(5, 10]","lstat_(10, 20]","lstat_(20, 40]"
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0
4,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0


In [7]:
# 去掉离群点
y = data_train['medv']

X_new = X_new[y != 50].values
y = y[y != 50]

## 选择超参数

In [8]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(nthread=7)

In [9]:
grid_params = {
    'max_depth': [4, 5, 6, 7],
    'learning_rate': np.linspace(0.03, 0.3, 10),
    'n_estimators': [100, 200]
}

In [10]:
from sklearn.model_selection import ShuffleSplit

cv_split = ShuffleSplit(n_splits=6, test_size=0.2)

In [11]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(
    xgbr,
    grid_params,
    cv=cv_split,
    scoring='neg_mean_squared_error',
    verbose=1)

In [12]:
%%time
grid.fit(X_new, y)

Fitting 6 folds for each of 80 candidates, totalling 480 fits
[CV] learning_rate=0.03, max_depth=4, n_estimators=100 ...............
[CV]  learning_rate=0.03, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.03, max_depth=4, n_estimators=100 ...............
[CV]  learning_rate=0.03, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.03, max_depth=4, n_estimators=100 ...............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  learning_rate=0.03, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.03, max_depth=4, n_estimators=100 ...............
[CV]  learning_rate=0.03, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.03, max_depth=4, n_estimators=100 ...............
[CV]  learning_rate=0.03, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.03, max_depth=4, n_estimators=100 ...............
[CV]  learning_rate=0.03, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.03, max_depth=4, n_estimators=200 ...............
[CV]  learning_rate=0.03, max_depth=4, n_estimators=200, total=   0.2s
[CV] learning_rate=0.03, max_depth=4, n_estimators=200 ...............
[CV]  learning_rate=0.03, max_depth=4, n_estimators=200, total=   0.2s
[CV] learning_rate=0.03, max_depth=4, n_estimators=200 ...............
[CV]  learning_rate=0.03, max_depth=4, n_estimators=200, total=   0.2s
[CV] learning_rate=0.03, max_depth=4, n_estimators=200 ...............
[CV]  

[CV]  learning_rate=0.06, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.06, max_depth=5, n_estimators=100 ...............
[CV]  learning_rate=0.06, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.06, max_depth=5, n_estimators=100 ...............
[CV]  learning_rate=0.06, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.06, max_depth=5, n_estimators=100 ...............
[CV]  learning_rate=0.06, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.06, max_depth=5, n_estimators=100 ...............
[CV]  learning_rate=0.06, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.06, max_depth=5, n_estimators=200 ...............
[CV]  learning_rate=0.06, max_depth=5, n_estimators=200, total=   0.2s
[CV] learning_rate=0.06, max_depth=5, n_estimators=200 ...............
[CV]  learning_rate=0.06, max_depth=5, n_estimators=200, total=   0.2s
[CV] learning_rate=0.06, max_depth=5, n_estimators=200 ...............
[CV]  

[CV]  learning_rate=0.09, max_depth=5, n_estimators=200, total=   0.2s
[CV] learning_rate=0.09, max_depth=6, n_estimators=100 ...............
[CV]  learning_rate=0.09, max_depth=6, n_estimators=100, total=   0.1s
[CV] learning_rate=0.09, max_depth=6, n_estimators=100 ...............
[CV]  learning_rate=0.09, max_depth=6, n_estimators=100, total=   0.1s
[CV] learning_rate=0.09, max_depth=6, n_estimators=100 ...............
[CV]  learning_rate=0.09, max_depth=6, n_estimators=100, total=   0.1s
[CV] learning_rate=0.09, max_depth=6, n_estimators=100 ...............
[CV]  learning_rate=0.09, max_depth=6, n_estimators=100, total=   0.1s
[CV] learning_rate=0.09, max_depth=6, n_estimators=100 ...............
[CV]  learning_rate=0.09, max_depth=6, n_estimators=100, total=   0.1s
[CV] learning_rate=0.09, max_depth=6, n_estimators=100 ...............
[CV]  learning_rate=0.09, max_depth=6, n_estimators=100, total=   0.1s
[CV] learning_rate=0.09, max_depth=6, n_estimators=200 ...............
[CV]  

...

[CV]  learning_rate=0.21000000000000002, max_depth=7, n_estimators=200, total=   0.2s
[CV] learning_rate=0.21000000000000002, max_depth=7, n_estimators=200 
[CV]  learning_rate=0.21000000000000002, max_depth=7, n_estimators=200, total=   0.2s
[CV] learning_rate=0.24000000000000002, max_depth=4, n_estimators=100 
[CV]  learning_rate=0.24000000000000002, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.24000000000000002, max_depth=4, n_estimators=100 
[CV]  learning_rate=0.24000000000000002, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.24000000000000002, max_depth=4, n_estimators=100 
[CV]  learning_rate=0.24000000000000002, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.24000000000000002, max_depth=4, n_estimators=100 
[CV]  learning_rate=0.24000000000000002, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.24000000000000002, max_depth=4, n_estimators=100 
[CV]  learning_rate=0.24000000000000002, max_depth=4, n_es

[CV]  learning_rate=0.27, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.27, max_depth=4, n_estimators=100 ...............
[CV]  learning_rate=0.27, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.27, max_depth=4, n_estimators=100 ...............
[CV]  learning_rate=0.27, max_depth=4, n_estimators=100, total=   0.1s
[CV] learning_rate=0.27, max_depth=4, n_estimators=200 ...............
[CV]  learning_rate=0.27, max_depth=4, n_estimators=200, total=   0.2s
[CV] learning_rate=0.27, max_depth=4, n_estimators=200 ...............
[CV]  learning_rate=0.27, max_depth=4, n_estimators=200, total=   0.2s
[CV] learning_rate=0.27, max_depth=4, n_estimators=200 ...............
[CV]  learning_rate=0.27, max_depth=4, n_estimators=200, total=   0.2s
[CV] learning_rate=0.27, max_depth=4, n_estimators=200 ...............
[CV]  learning_rate=0.27, max_depth=4, n_estimators=200, total=   0.2s
[CV] learning_rate=0.27, max_depth=4, n_estimators=200 ...............
[CV]  

[CV] . learning_rate=0.3, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.3, max_depth=5, n_estimators=100 ................
[CV] . learning_rate=0.3, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.3, max_depth=5, n_estimators=100 ................
[CV] . learning_rate=0.3, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.3, max_depth=5, n_estimators=100 ................
[CV] . learning_rate=0.3, max_depth=5, n_estimators=100, total=   0.1s
[CV] learning_rate=0.3, max_depth=5, n_estimators=200 ................
[CV] . learning_rate=0.3, max_depth=5, n_estimators=200, total=   0.2s
[CV] learning_rate=0.3, max_depth=5, n_estimators=200 ................
[CV] . learning_rate=0.3, max_depth=5, n_estimators=200, total=   0.2s
[CV] learning_rate=0.3, max_depth=5, n_estimators=200 ................
[CV] . learning_rate=0.3, max_depth=5, n_estimators=200, total=   0.2s
[CV] learning_rate=0.3, max_depth=5, n_estimators=200 ................
[CV] .

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  1.3min finished


GridSearchCV(cv=ShuffleSplit(n_splits=6, random_state=None, test_size=0.2, train_size=None),
       error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=7, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [4, 5, 6, 7], 'learning_rate': array([0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27, 0.3 ]), 'n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=2)

In [17]:
grid.best_params_

{'learning_rate': 0.09, 'max_depth': 4, 'n_estimators': 200}

In [18]:
grid.best_score_

-8.233212548914441

In [21]:
np.sqrt(-grid.best_score_)

2.869357515004786

因为训练数据有限，所以用最佳超参数把全部的训练数据集拿出来再训练一次。

In [25]:
xgbr_best = XGBRegressor(nthread=7, **grid.best_params_)

xgbr_best.fit(X_new, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.09, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=7, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [27]:
data_test = pd.read_csv('../input/test.csv')
X_test = data_test.drop(['ID'], axis=1)
X_test = dataProcessing(X_test).values
y_pred = xgbr_best.predict(X_test)

In [28]:
res = pd.DataFrame({'ID': data_test.ID, 'medv': y_pred})
res.to_csv('../output/xgbr_boston_submission1.csv', index=False)

# 3.08648